In [2]:
#!/usr/bin/env python3
"""
Filter2D 高性能图像处理 - PYNQ实现（NPY批处理版本）
支持处理.npy格式的批量图像数据
"""

import numpy as np
import time
from pynq import Overlay, allocate
import cv2
import os

class Filter2D_Accelerator:
    """Filter2D硬件加速器封装类"""
    
    # 硬件配置参数
    NPIX = 4  # 每次处理4个像素
    PIXEL_WIDTH = 24  # 每像素24位(RGB各8位)
    
    def __init__(self, bitstream_path):
        """
        初始化加速器
        
        Args:
            bitstream_path: .bit文件路径
        """
        print("正在加载比特流...")
        self.overlay = Overlay(bitstream_path)
        
        # 获取IP核和DMA
        self.dma = self.overlay.axi_dma_0
        self.filter2d = self.overlay.Filter2D_accel_0
        
        print(f"✓ 比特流加载成功")
        print(f"✓ DMA地址: 0x{self.dma.mmio.base_addr:X}")
        print(f"✓ Filter2D地址: 0x{self.filter2d.mmio.base_addr:X}")
        
    def process_image(self, input_image, verbose=True):
        """
        处理单张图像
        
        Args:
            input_image: OpenCV格式的BGR图像 (numpy array)
            verbose: 是否打印详细信息
            
        Returns:
            处理后的图像 (numpy array)
        """
        # 验证输入
        if input_image is None or input_image.size == 0:
            raise ValueError("输入图像无效")
            
        height, width = input_image.shape[:2]
        
        # 检查是否为RGB图像
        if len(input_image.shape) != 3 or input_image.shape[2] != 3:
            raise ValueError(f"输入必须是3通道RGB图像，当前shape: {input_image.shape}")
        
        # 宽度对齐检查（必须是NPIX的倍数）
        if width % self.NPIX != 0:
            raise ValueError(f"图像宽度必须是{self.NPIX}的倍数，当前宽度: {width}")
        
        if verbose:
            print(f"\n{'='*60}")
            print(f"处理图像: {width}x{height}")
            print(f"{'='*60}")
        
        # 计算传输大小
        image_size_bytes = height * width * 3
        
        # 分配物理连续内存
        if verbose:
            print("分配DMA缓冲区...")
        input_buffer = allocate(shape=(image_size_bytes,), dtype=np.uint8)
        output_buffer = allocate(shape=(image_size_bytes,), dtype=np.uint8)
        
        # 将图像数据拷贝到输入缓冲区
        # OpenCV使用BGR格式，需要转换为RGB
        rgb_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
        input_buffer[:] = rgb_image.flatten()
        
        # 刷新缓存，确保数据写入DDR
        input_buffer.flush()
        
        # 配置Filter2D参数
        if verbose:
            print(f"配置加速器参数...")
        self.filter2d.write(0x10, height)  # 偏移0x10: height参数
        self.filter2d.write(0x18, width)   # 偏移0x18: width参数
        
        # 启动加速器
        if verbose:
            print("启动加速器...")
        self.filter2d.write(0x00, 0x01)  # AP_START
        
        # 启动DMA传输
        start_time = time.time()
        if verbose:
            print("启动DMA传输...")
        
        # 必须先启动接收（S2MM），再启动发送（MM2S）
        self.dma.recvchannel.transfer(output_buffer)
        self.dma.sendchannel.transfer(input_buffer)
        
        # 等待DMA传输完成
        if verbose:
            print("等待DMA传输完成...")
        self.dma.sendchannel.wait()
        self.dma.recvchannel.wait()
        
        # 等待加速器完成
        if verbose:
            print("等待加速器完成...")
        while (self.filter2d.read(0x00) & 0x02) == 0:  # 检查AP_DONE位
            pass
        
        transfer_time = time.time() - start_time
        
        # 使缓存无效，从DDR读取最新数据
        output_buffer.invalidate()
        
        # 将输出数据重塑为图像
        output_image = np.reshape(output_buffer, (height, width, 3))
        
        # 转换回BGR格式用于OpenCV显示
        output_image = cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR)
        
        # 释放缓冲区
        del input_buffer
        del output_buffer
        
        # 计算性能
        if verbose:
            throughput_mbps = (image_size_bytes * 2 * 8) / (transfer_time * 1e6)
            fps = 1.0 / transfer_time if transfer_time > 0 else 0
            
            print(f"\n{'='*60}")
            print(f"✓ 处理完成")
            print(f"  传输时间: {transfer_time*1000:.2f} ms")
            print(f"  吞吐量: {throughput_mbps:.2f} Mbps")
            print(f"  帧率: {fps:.2f} FPS")
            print(f"{'='*60}\n")
        
        return output_image, transfer_time
    
    def process_npy_volume(self, npy_path, output_path=None, save_format='npy'):
        """
        处理npy格式的批量图像数据
        
        Args:
            npy_path: 输入npy文件路径
            output_path: 输出文件路径（如果为None，自动生成）
            save_format: 保存格式 ('npy' 或 'individual')
                - 'npy': 保存为单个npy文件
                - 'individual': 保存为单独的图像文件
            
        Returns:
            处理后的图像数组
        """
        print(f"\n{'='*60}")
        print(f"批量处理NPY文件")
        print(f"{'='*60}")
        
        # 读取npy文件
        print(f"\n读取文件: {npy_path}")
        try:
            images = np.load(npy_path)
        except Exception as e:
            print(f"❌ 读取文件失败: {e}")
            return None
        
        print(f"✓ 文件读取成功")
        print(f"  数组形状: {images.shape}")
        print(f"  数据类型: {images.dtype}")
        print(f"  数值范围: [{images.min()}, {images.max()}]")
        
        # 验证数据格式
        if len(images.shape) != 4:
            print(f"❌ 错误: 期望4维数组 (N, H, W, C)，实际: {images.shape}")
            return None
        
        num_images, height, width, channels = images.shape
        
        if channels != 3:
            print(f"❌ 错误: 期望3通道图像，实际: {channels}")
            return None
        
        if width % self.NPIX != 0:
            print(f"❌ 错误: 图像宽度必须是{self.NPIX}的倍数，实际: {width}")
            return None
        
        print(f"\n准备处理 {num_images} 张图像 ({width}x{height})")
        
        # 创建输出数组
        output_images = np.zeros_like(images)
        
        # 处理统计
        total_start = time.time()
        processing_times = []
        
        # 逐张处理
        for i in range(num_images):
            # 获取当前图像（假设输入是RGB格式）
            img = images[i]
            
            # 转换为BGR格式（OpenCV格式）
            img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            
            # 处理图像
            try:
                output_img, proc_time = self.process_image(img_bgr, verbose=False)
                processing_times.append(proc_time)
                
                # 转换回RGB格式
                output_img_rgb = cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB)
                output_images[i] = output_img_rgb
                
                # 打印进度
                progress = (i + 1) / num_images * 100
                avg_time = np.mean(processing_times) * 1000
                remaining = (num_images - i - 1) * np.mean(processing_times)
                
                print(f"进度: {i+1}/{num_images} ({progress:.1f}%) | "
                      f"平均: {avg_time:.2f}ms/帧 | "
                      f"剩余: {remaining:.1f}s", end='\r')
                
            except Exception as e:
                print(f"\n❌ 处理第 {i+1} 张图像时失败: {e}")
                continue
        
        print()  # 换行
        
        total_time = time.time() - total_start
        avg_fps = num_images / total_time if total_time > 0 else 0
        
        print(f"\n{'='*60}")
        print(f"✓ 批量处理完成")
        print(f"  总图像数: {num_images}")
        print(f"  总时间: {total_time:.2f} s")
        print(f"  平均处理时间: {np.mean(processing_times)*1000:.2f} ms/帧")
        print(f"  平均帧率: {avg_fps:.2f} FPS")
        print(f"  总吞吐量: {(num_images * height * width * 3 * 2 * 8) / (total_time * 1e6):.2f} Mbps")
        print(f"{'='*60}\n")
        
        # 保存结果
        if output_path is None:
            base_name = os.path.splitext(npy_path)[0]
            output_path = f"{base_name}_processed.npy"
        
        if save_format == 'npy':
            print(f"保存处理结果到: {output_path}")
            np.save(output_path, output_images)
            print(f"✓ 已保存为NPY文件")
            
        elif save_format == 'individual':
            # 创建输出目录
            output_dir = os.path.splitext(output_path)[0]
            os.makedirs(output_dir, exist_ok=True)
            
            print(f"保存单独图像到目录: {output_dir}/")
            for i in range(num_images):
                img_path = os.path.join(output_dir, f"image_{i:04d}.png")
                # 转换为BGR保存
                img_bgr = cv2.cvtColor(output_images[i], cv2.COLOR_RGB2BGR)
                cv2.imwrite(img_path, img_bgr)
            print(f"✓ 已保存 {num_images} 张图像")
        
        return output_images


def pad_image_to_npix(image, npix=4):
    """
    将图像宽度填充到NPIX的倍数
    
    Args:
        image: 输入图像
        npix: 像素并行度
        
    Returns:
        填充后的图像和原始宽度
    """
    height, width = image.shape[:2]
    
    if width % npix == 0:
        return image, width
    
    # 计算需要填充的宽度
    pad_width = npix - (width % npix)
    padded_image = cv2.copyMakeBorder(image, 0, 0, 0, pad_width, 
                                      cv2.BORDER_REPLICATE)
    
    print(f"图像宽度从 {width} 填充到 {width + pad_width}")
    return padded_image, width


def process_npy_file():
    """处理NPY文件的主函数"""
    
    # ============================================
    # 配置参数
    # ============================================
    BITSTREAM_PATH = "sys5.bit"  # 修改为你的.bit文件路径
    NPY_FILE_PATH = "images_2_volume.npy"  # NPY文件路径
    OUTPUT_NPY_PATH = "images_2_volume_processed.npy"  # 输出NPY文件路径
    
    # 保存格式选项:
    # 'npy' - 保存为单个npy文件
    # 'individual' - 保存为单独的图像文件
    SAVE_FORMAT = 'npy'
    
    # ============================================
    # 初始化加速器
    # ============================================
    print(f"Filter2D NPY批量处理")
    print(f"{'='*60}\n")
    
    try:
        accelerator = Filter2D_Accelerator(BITSTREAM_PATH)
    except Exception as e:
        print(f"❌ 初始化失败: {e}")
        return
    
    # ============================================
    # 处理NPY文件
    # ============================================
    try:
        output_images = accelerator.process_npy_volume(
            NPY_FILE_PATH, 
            OUTPUT_NPY_PATH,
            save_format=SAVE_FORMAT
        )
        
        if output_images is not None:
            print(f"\n✓ 所有处理完成!")
            print(f"  输出文件: {OUTPUT_NPY_PATH}")
            
    except Exception as e:
        print(f"❌ 处理失败: {e}")
        import traceback
        traceback.print_exc()


def main_single_image():
    """单张图像处理"""
    
    # ============================================
    # 配置参数
    # ============================================
    BITSTREAM_PATH = "sys5.bit"  # 修改为你的.bit文件路径
    INPUT_IMAGE_PATH = "input.jpg"  # 输入图像路径
    OUTPUT_IMAGE_PATH = "output.jpg"  # 输出图像路径
    
    # ============================================
    # 初始化加速器
    # ============================================
    try:
        accelerator = Filter2D_Accelerator(BITSTREAM_PATH)
    except Exception as e:
        print(f"❌ 初始化失败: {e}")
        return
    
    # ============================================
    # 读取输入图像
    # ============================================
    print(f"读取图像: {INPUT_IMAGE_PATH}")
    input_image = cv2.imread(INPUT_IMAGE_PATH)
    
    if input_image is None:
        print(f"❌ 无法读取图像: {INPUT_IMAGE_PATH}")
        return
    
    print(f"✓ 图像读取成功: {input_image.shape[1]}x{input_image.shape[0]}")
    
    # ============================================
    # 处理图像
    # ============================================
    # 检查并填充图像
    padded_image, original_width = pad_image_to_npix(input_image, npix=4)
    
    try:
        # 处理图像
        output_image, _ = accelerator.process_image(padded_image, verbose=True)
        
        # 如果进行了填充，裁剪回原始宽度
        if padded_image.shape[1] != original_width:
            output_image = output_image[:, :original_width]
            print(f"裁剪输出图像到原始宽度: {original_width}")
        
        # 保存输出图像
        cv2.imwrite(OUTPUT_IMAGE_PATH, output_image)
        print(f"✓ 输出图像已保存: {OUTPUT_IMAGE_PATH}")
        
    except Exception as e:
        print(f"❌ 处理失败: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    # 选择测试模式
    print("Filter2D PYNQ 加速器测试")
    print("=" * 60)
    print("1. NPY批量图像处理 (推荐用于images_2_volume.npy)")
    print("2. 单张图像处理")
    print("=" * 60)
    
    mode = input("选择模式 (1/2): ").strip()
    
    if mode == "1":
        process_npy_file()
    elif mode == "2":
        main_single_image()
    else:
        print("无效选择，执行默认NPY批量处理")
        process_npy_file()

Filter2D PYNQ 加速器测试
1. NPY批量图像处理 (推荐用于images_2_volume.npy)
2. 单张图像处理


选择模式 (1/2):  1


Filter2D NPY批量处理

正在加载比特流...


✓ 比特流加载成功
✓ DMA地址: 0xA0000000
✓ Filter2D地址: 0xA0010000

批量处理NPY文件

读取文件: images_2_volume.npy
✓ 文件读取成功
  数组形状: (108, 512, 512, 3)
  数据类型: uint8
  数值范围: [0, 255]

准备处理 108 张图像 (512x512)
进度: 108/108 (100.0%) | 平均: 1.12ms/帧 | 剩余: 0.0s

✓ 批量处理完成
  总图像数: 108
  总时间: 1.27 s
  平均处理时间: 1.12 ms/帧
  平均帧率: 84.84 FPS
  总吞吐量: 1067.49 Mbps

保存处理结果到: images_2_volume_processed.npy
✓ 已保存为NPY文件

✓ 所有处理完成!
  输出文件: images_2_volume_processed.npy
